In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from tslearn.metrics import cdist_dtw, dtw
from scipy.signal import correlate

/tmp/ipykernel_673051/3346051274.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/giovani/anaconda3/envs/wavelet_code/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
def dtw_similarity(multi_ts, r = 1):
    assert multi_ts.ndim == 2
    k = multi_ts.shape[1]
    out = np.zeros((k, k))
    for i in range(k):
        for j in range(i, k):
            d = dtw(multi_ts[:, i], multi_ts[:, j], global_constraint="sakoe_chiba", sakoe_chiba_radius=r)
            out[i, j] = d
            out[j, i] = out[i, j]	
    return out

In [3]:
def save_dtw_dist_matrix(poly_division, time_interval):
    # load and normalize signals

    df = pd.read_csv(f"data/polygon_data/{poly_division}_{time_interval}.csv")
    features = df.columns[2:]
    signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
    for i in range(signal_multi.shape[2]):
        mu = np.mean(signal_multi[:, :, i])
        sigma = np.std(signal_multi[:, :, i])

        if sigma == 0:
            sigma = 1
        signal_multi[:, :, i] = (signal_multi[:, :, i] - mu) / sigma

    # calculate dtw distance matrix
    geral_dist = np.zeros((signal_multi.shape[2], signal_multi.shape[2]))
    for i in tqdm(range(signal_multi.shape[0])):
        sample_dist = dtw_similarity(signal_multi[i], r = 5)
        geral_dist += sample_dist   

    geral_dist /= signal_multi.shape[0] 

    df = []
    for i in range(geral_dist.shape[0]):
        for j in range(i, geral_dist.shape[1]):
            df.append({
                "row": features[i],
                "column": features[j],
                "value": geral_dist[i, j]
            })
    df = pd.DataFrame(df)

    df.to_csv(f"data/similarity_matrix/{poly_division}_{time_interval}.csv", index=False)

In [4]:
save_dtw_dist_matrix("SpCenterCensus5k", "Period1")

/tmp/ipykernel_673051/3312879201.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
100%|██████████| 5000/5000 [00:16<00:00, 299.52it/s]


In [5]:
save_dtw_dist_matrix("SpCenterCensus5k", "Period2")

/tmp/ipykernel_673051/3312879201.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
100%|██████████| 5000/5000 [00:08<00:00, 585.89it/s]


In [4]:
for time_interval in ["Month", "Day", "3days", "5days"]:
    for poly_division in ["SpGrid", "SpDistricts", "SpCenterCensus5k", "SpCenterCensus2k"]:
        save_dtw_dist_matrix(poly_division, time_interval)

/tmp/ipykernel_309271/3312879201.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
100%|██████████| 2500/2500 [00:04<00:00, 594.03it/s]
/tmp/ipykernel_309271/3312879201.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(l

In [4]:
for time_interval in ["Day", "3days", "5days"]:
    for poly_division in ["NYBlocks"]:
        save_dtw_dist_matrix(poly_division, time_interval)

/tmp/ipykernel_371380/3312879201.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
100%|██████████| 1168/1168 [00:02<00:00, 568.61it/s]
/tmp/ipykernel_371380/3312879201.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(l

In [4]:
save_dtw_dist_matrix("BLACities", "Year")

/tmp/ipykernel_532745/3312879201.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  signal_multi = np.stack(df.groupby("id_poly").apply(lambda x: x.sort_values("date").drop(columns=["id_poly", "date"]).values).values)
100%|██████████| 773/773 [00:03<00:00, 245.36it/s]
